In [ ]:
# Step 1: Install Required Libraries
!pip install pandas google-cloud-bigquery google-cloud-storage

In [ ]:
# Step 2: Upload the service account JSON file
from google.colab import files
uploaded = files.upload()

In [6]:
# Step 3: Authenticate with the service account
import os
import json

# Replace 'your_service_account.json' with the filename of the uploaded JSON key
service_account_info = json.load(open('sumup_case_sa.json'))
project_id = service_account_info['project_id']

from google.oauth2 import service_account
credentials = service_account.Credentials.from_service_account_info(service_account_info)

from google.cloud import bigquery
from google.cloud import storage

bq_client = bigquery.Client(credentials=credentials, project=project_id)
storage_client = storage.Client(credentials=credentials, project=project_id)

In [7]:
# Step 4: Convert Excel Files to CSV
import pandas as pd

excel_files = ['store.xlsx', 'device.xlsx', 'transaction.xlsx']
csv_files = ['stores.csv', 'devices.csv', 'transactions.csv']

for excel_file, csv_file in zip(excel_files, csv_files):
    df = pd.read_excel(excel_file)
    df.to_csv(csv_file, index=False)

In [8]:
# Step 5: Upload CSV Files to Google Cloud Storage
bucket_name = "sumup_case"
bucket = storage_client.bucket(bucket_name)

for file in csv_files:
    blob = bucket.blob(file)
    blob.upload_from_filename(file)


In [10]:
# Step 6: Load CSV Data into BigQuery
def load_data_from_gcs_to_bq(dataset_name, table_name, gcs_uri):
    dataset_ref = bq_client.dataset(dataset_name)
    table_ref = dataset_ref.table(table_name)

    job_config = bigquery.LoadJobConfig(
        autodetect=True,
        source_format=bigquery.SourceFormat.CSV,
        skip_leading_rows=1,
        write_disposition=bigquery.WriteDisposition.WRITE_TRUNCATE
    )

    load_job = bq_client.load_table_from_uri(
        gcs_uri,
        table_ref,
        job_config=job_config
    )
    load_job.result()  # Waits for the job to complete

    print(f"Loaded {load_job.output_rows} rows into {dataset_name}:{table_name}")

dataset_name = 'SumUp_Case'
bucket_uri = f"gs://{bucket_name}/"

load_data_from_gcs_to_bq(dataset_name, 'stores', f"{bucket_uri}stores.csv")
load_data_from_gcs_to_bq(dataset_name, 'devices', f"{bucket_uri}devices.csv")
load_data_from_gcs_to_bq(dataset_name, 'transactions', f"{bucket_uri}transactions.csv")


Loaded 100 rows into SumUp_Case:stores
Loaded 200 rows into SumUp_Case:devices
Loaded 1500 rows into SumUp_Case:transactions
